In [31]:
# import langchain olama
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3"
model = Ollama(model=MODEL)
# embedding is specific to the model 
embeddings = OllamaEmbeddings(model=MODEL)

In [33]:
from langchain_community.document_loaders import PyPDFLoader

pdf_files = ["choreo.pdf", "choreo_2.pdf", "choreo_3.pdf"]

pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    data = loader.load_and_split()
    pages.extend(data)


In [35]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
find the answer, just say "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="here is some context", question="Here is a question")



'\nAnswer the question based on the context below. If you can\'t \nfind the answer, just say "I don\'t know".\n\nContext: here is some context\n\nQuestion: Here is a question\n'

In [36]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [37]:
retriever.invoke("what is choreo ")

[Document(page_content='07/06/2024, 09:15 Choreo - Stop Building Platforms. Just Add Developers Instead.\nhttps://wso2.com/choreo/ 9/9', metadata={'source': 'choreo_2.pdf', 'page': 8}),
 Document(page_content='We built Choreo so you don’t have to.\nGet Started Free\nContact Us\nQuote sign5\nHere’s What Our Users Say\nOur users are here to stay! Try it out for yourself and give us feedback on our\nDISCORD\n07/06/2024, 09:15 Choreo - Stop Building Platforms. Just Add Developers Instead.\nhttps://wso2.com/choreo/ 4/9', metadata={'source': 'choreo_2.pdf', 'page': 3}),
 Document(page_content='Multi-Environment CI/CD\nTreat any API as code and CI/CD to your heart’s content across multiple deployment environments. Integrate\nverification, testing, or documentation systems to the build.\nMulti-Cloud and Regions\nDeploy your APIs across clouds (Azure, AWS, GCP, your own Kubernetes) in any region of choice.\nAPI Sandbox\nFacilitate developer adoption with a sandbox endpoint for your APIs for exp

In [40]:
from operator import itemgetter

agent = (
  {"question": itemgetter("question"), "context": itemgetter("question") | retriever}
  | prompt
  | model
)

chain.invoke({"question": "What is choreo?"})

'Based on the context, it appears that Choreo is a platform or tool for building and managing APIs. It allows users to create managed, secured, proxy APIs for their existing private or SaaS backend services, and provides features such as GitHub integration, multiple environments, and cloud/region support. Additionally, Choreo has a quote on its page that says "We built Choreo so you don’t have to." This suggests that Choreo is designed to simplify the process of building APIs for developers.'